# Capítulo 1: Introdução ao SQL no DuckDB

Este notebook demonstra os conceitos básicos de SQL usando DuckDB:
- Criação de tabelas
- Inserção de dados
- Comandos SELECT, WHERE, ORDER BY, DISTINCT
- JOINs (INNER, LEFT OUTER)
- Funções de agregação (GROUP BY, HAVING)
- UPDATE e DELETE

In [ ]:
import duckdb

## 1.1 Criando Tabelas e Inserindo Dados

In [ ]:
# Criar conexão em memória
con = duckdb.connect()

# Criar tabela weather
con.execute("""
    CREATE TABLE weather (
        city VARCHAR,
        temp_lo INTEGER,      -- temperatura mínima
        temp_hi INTEGER,      -- temperatura máxima
        prcp FLOAT,           -- precipitação
        date DATE
    )
""")

# Criar tabela cities
con.execute("""
    CREATE TABLE cities (
        name VARCHAR,
        lat DECIMAL,
        lon DECIMAL
    )
""")

print("✓ Tabelas 'weather' e 'cities' criadas com sucesso!")

In [ ]:
# Inserir dados na tabela weather
con.execute("INSERT INTO weather VALUES ('San Francisco', 46, 50, 0.25, '1994-11-27')")
con.execute("INSERT INTO weather (city, temp_lo, temp_hi, prcp, date) VALUES ('San Francisco', 43, 57, 0.0, '1994-11-29')")
con.execute("INSERT INTO weather (date, city, temp_hi, temp_lo) VALUES ('1994-11-29', 'Hayward', 54, 37)")
con.execute("INSERT INTO weather VALUES ('Los Angeles', 50, 75, 0.0, '1994-11-28')")
con.execute("INSERT INTO weather VALUES ('Seattle', 35, 45, 0.5, '1994-11-27')")

# Inserir dados na tabela cities
con.execute("INSERT INTO cities VALUES ('San Francisco', 37.7749, -122.4194)")
con.execute("INSERT INTO cities VALUES ('Hayward', 37.6688, -122.0808)")
con.execute("INSERT INTO cities VALUES ('Los Angeles', 34.0522, -118.2437)")

print("✓ Dados inseridos nas tabelas 'weather' e 'cities'")

## 1.2 SELECT Simples

In [ ]:
# Selecionar todas as colunas
con.sql("SELECT * FROM weather").show()

In [ ]:
# Selecionar colunas específicas
con.sql("SELECT city, temp_lo, temp_hi, date FROM weather").show()

## 1.3 Expressões e Aliases

In [ ]:
# Calcular temperatura média
con.sql("""
    SELECT city, (temp_hi + temp_lo) / 2 AS temp_avg, date
    FROM weather
""").show()

## 1.4 Comando WHERE

In [ ]:
# Filtrar por cidade e precipitação
con.sql("""
    SELECT * FROM weather
    WHERE city = 'San Francisco' AND prcp > 0.0
""").show()

## 1.5 ORDER BY

In [ ]:
# Ordenar por cidade
con.sql("SELECT * FROM weather ORDER BY city").show()

In [ ]:
# Ordenar por múltiplas colunas
con.sql("SELECT * FROM weather ORDER BY city, temp_lo").show()

## 1.6 DISTINCT

In [ ]:
# Obter cidades únicas
con.sql("SELECT DISTINCT city FROM weather ORDER BY city").show()

## 1.7 JOINS

In [ ]:
# Inner Join (sintaxe tradicional)
con.sql("""
    SELECT weather.city, temp_lo, temp_hi, prcp, date, lon, lat
    FROM weather, cities
    WHERE city = name
""").show()

In [ ]:
# Inner Join (sintaxe explícita)
con.sql("""
    SELECT weather.city, temp_lo, temp_hi, lat, lon
    FROM weather
    INNER JOIN cities ON weather.city = cities.name
""").show()

In [ ]:
# Left Outer Join (mostra todas as cidades, mesmo sem coordenadas)
con.sql("""
    SELECT weather.city, temp_lo, temp_hi, lat, lon
    FROM weather
    LEFT OUTER JOIN cities ON weather.city = cities.name
""").show()

## 1.8 Funções de Agregação

In [ ]:
# Temperatura mínima mais alta
con.sql("SELECT max(temp_lo) FROM weather").show()

In [ ]:
# Cidade com a temperatura mínima mais alta
con.sql("""
    SELECT city FROM weather
    WHERE temp_lo = (SELECT max(temp_lo) FROM weather)
""").show()

## 1.9 GROUP BY

In [ ]:
# Temperatura mínima máxima por cidade
con.sql("""
    SELECT city, max(temp_lo) as max_temp_lo
    FROM weather
    GROUP BY city
    ORDER BY city
""").show()

## 1.10 HAVING

In [ ]:
# Cidades com temperatura mínima máxima < 45
con.sql("""
    SELECT city, max(temp_lo) as max_temp_lo
    FROM weather
    GROUP BY city
    HAVING max(temp_lo) < 45
""").show()

## 1.11 LIKE com GROUP BY

In [ ]:
# Cidades que começam com 'S'
con.sql("""
    SELECT city, max(temp_lo) as max_temp_lo
    FROM weather
    WHERE city LIKE 'S%'
    GROUP BY city
    HAVING max(temp_lo) < 50
""").show()

## 1.12 UPDATE

In [ ]:
# Antes do UPDATE
print("Antes do UPDATE:")
con.sql("SELECT city, temp_hi, temp_lo, date FROM weather WHERE date > '1994-11-28'").show()

# Executar UPDATE
con.execute("""
    UPDATE weather
    SET temp_hi = temp_hi - 2, temp_lo = temp_lo - 2
    WHERE date > '1994-11-28'
""")

# Depois do UPDATE
print("\nDepois do UPDATE (reduzindo 2°F):")
con.sql("SELECT city, temp_hi, temp_lo, date FROM weather WHERE date > '1994-11-28'").show()

## 1.13 DELETE

In [ ]:
# Antes do DELETE
print("Registros antes do DELETE:")
con.sql("SELECT COUNT(*) as total FROM weather").show()

# Executar DELETE
con.execute("DELETE FROM weather WHERE city = 'Hayward'")

# Depois do DELETE
print("\nRegistros depois do DELETE (removido Hayward):")
con.sql("SELECT COUNT(*) as total FROM weather").show()

print("\nRegistros restantes:")
con.sql("SELECT city FROM weather ORDER BY city").show()

## Exercícios Práticos

### Exercício 1: Criar tabela 'produtos'

In [ ]:
con.execute("""
    CREATE TABLE produtos (
        nome VARCHAR,
        categoria VARCHAR,
        preco DECIMAL(10, 2),
        estoque INTEGER
    )
""")
print("✓ Tabela 'produtos' criada!")

### Exercício 2: Inserir 5 produtos

In [ ]:
produtos_data = [
    ('Notebook Dell', 'Eletrônicos', 3500.00, 10),
    ('Mouse Logitech', 'Eletrônicos', 89.90, 50),
    ('Cadeira Gamer', 'Móveis', 1200.00, 15),
    ('Mesa Escritório', 'Móveis', 800.00, 8),
    ('Teclado Mecânico', 'Eletrônicos', 450.00, 25)
]

for produto in produtos_data:
    con.execute("INSERT INTO produtos VALUES (?, ?, ?, ?)", produto)

print("✓ 5 produtos inseridos!")
con.sql("SELECT * FROM produtos").show()

### Exercício 3: Produto mais caro

In [ ]:
con.sql("""
    SELECT nome, preco
    FROM produtos
    WHERE preco = (SELECT MAX(preco) FROM produtos)
""").show()

### Exercício 4: Preço médio por categoria

In [ ]:
con.sql("""
    SELECT categoria, AVG(preco) as preco_medio
    FROM produtos
    GROUP BY categoria
    ORDER BY categoria
""").show()

### Exercício 5: Categorias com preço médio > 50

In [ ]:
con.sql("""
    SELECT categoria, AVG(preco) as preco_medio
    FROM produtos
    GROUP BY categoria
    HAVING AVG(preco) > 50
    ORDER BY preco_medio DESC
""").show()

## Conclusão

Neste capítulo você aprendeu:
- Como criar tabelas e inserir dados
- Comandos SELECT fundamentais (WHERE, ORDER BY, DISTINCT)
- Como fazer joins entre tabelas
- Funções de agregação e agrupamento (GROUP BY, HAVING)
- Como atualizar e deletar dados
- Principais diferenças entre DuckDB e PostgreSQL

In [ ]:
# Fechar conexão
con.close()
print("✓ Conexão fechada")